# General instructions

The goal of the project is to materialize a set of **exploratory workloads** over a real-world, large-scale,  open-domain KG: [WikiData](https://www.wikidata.org/wiki/Wikidata:Main_Page)

An exploratory workload  is composed by a set of queries, where each query is related to the information obtained previously.

An exploratory workload starts with a usually vague, open ended question, and does not assume the person issuing the workload has a clear understanding of the data contained in the target database or its structure.

Remeber that:

1. All the queries must run in the python notebook
2. You can use classes and properties only if you find them via a SPARQL query that must be present in the notebook
3. You do not delete useless queries. Keep everything that is synthatically valid 

```
?p <http://schema.org/name> ?name .
```
    
    is the BGP returning a human-readable name of a property or a class in Wikidata.
    
    

In [1]:
## SETUP used later

from SPARQLWrapper import SPARQLWrapper, JSON


prefixString = """
##-f79a19311a-##
PREFIX wd: <http://www.wikidata.org/entity/> 
PREFIX wdt: <http://www.wikidata.org/prop/direct/> 
PREFIX sc: <http://schema.org/>
"""

# select and construct queries
def run_query(queryString,verbose = True):
    to_run = prefixString + "\n" + queryString

    sparql = SPARQLWrapper("http://a256-gc1-03.srv.aau.dk:5820/sparql")
    sparql.setTimeout(300)
    sparql.setReturnFormat(JSON)
    sparql.setQuery(to_run)

    try :
        results = sparql.query()
        json_results = results.convert()
        if len(json_results['results']['bindings'])==0:
            print("Empty")
            return []
        array = []
        for bindings in json_results['results']['bindings']:
            app =  [ (var, value['value'])  for var, value in bindings.items() ] 
            if verbose:
                print( app)
            array.append(app)
        if verbose:
            print(len(array))
        return array

    except Exception as e :
        print("The operation failed", e)
    
# ASk queries
def run_ask_query(queryString):
    to_run = prefixString + "\n" + queryString

    sparql = SPARQLWrapper("http://a256-gc1-03.srv.aau.dk:5820/sparql")
    sparql.setTimeout(300)
    sparql.setReturnFormat(JSON)
    sparql.setQuery(to_run)

    try :
        return sparql.query().convert()

    except Exception as e :
        print("The operation failed", e)

# Movie Workflow Series ("Sherlock Holmes explorative search") 


Consider the following exploratory scenario:


> Investigate the canon of Sherlock Holmes (see https://en.wikipedia.org/wiki/Canon_of_Sherlock_Holmes) and which are the films and tv series related to the canon. We are also interested in the Emmy Awards and hiw many awards each film won. 


## Useful URIs for the current workflow
The following are given:

| IRI           | Description   | Role      |
| -----------   | -----------   |-----------|
| `wdt:P1647`   | subproperty   | predicate |
| `wdt:P31`     | instance of   | predicate |
| `wdt:P106`    | profession    | predicate | 
| `wdt:P279`    | subclass      | predicate |
| `wd:Q35610` | Arthur Conan Doyle        | node |



Also consider

```
wd:Q35610 ?p ?obj .
```

is the BGP to retrieve all **properties of Arthur Conan Doyle**

Please consider that when you return a resource, you should return the IRI and the label of the resource. In particular, when the task require you to identify a BGP the result set must always be a list of couples IRI - label.

The workload should:


1. Identify the BGP for canon of Sherlock Holmes (see https://en.wikipedia.org/wiki/Canon_of_Sherlock_Holmes)

2. Identify the BGP for film

3. Identify the BGP for television series

4. Return the year of publication (take the first one if there are more publication dates) for each film and television series related to the canon of Sherlock Holmes (the result set must be a list of triples film/tv series IRI, label and year).

5. Find who acted in more than one film or tv series related to the canon of Sherlock Holmes (the result set must be a list of triples actor/actress IRI, label and #film).

6. Identify the BGP for Primetime Emmy Award

7. Return all the types of Primetime Emmy Award (the result set must be a list of couples award IRI and label).

8. For each film and television series related to the canon of Sherlock Holmes, return the number of Primetime Emmy Awards they won (return also the objects who did not win an Award) (the result set must be a list of triples film/tv series IRI, label and #award).

## Task 1
Identify the BGP for canon of Sherlock Holmes (see https://en.wikipedia.org/wiki/Canon_of_Sherlock_Holmes)

In [2]:
# query example
queryString = """
SELECT DISTINCT ?p ?name
WHERE {
   # bind something
   wd:Q35610 ?p ?obj .
   # get the label
   ?p sc:name ?name.
}
LIMIT 20
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P4928'), ('name', 'Ricorso author ID')]
[('p', 'http://www.wikidata.org/prop/direct/P5714'), ('name', 'Tor.com author ID')]
[('p', 'http://www.wikidata.org/prop/direct/P9419'), ('name', 'personal library at')]
[('p', 'http://www.wikidata.org/prop/direct/P5470'), ('name', 'Scottish Poetry Library poet ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1005'), ('name', 'Portuguese National Library ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1006'), ('name', 'Nationale Thesaurus voor Auteurs ID')]
[('p', 'http://www.wikidata.org/prop/direct/P101'), ('name', 'field of work')]
[('p', 'http://www.wikidata.org/prop/direct/P1015'), ('name', 'NORAF ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1017'), ('name', 'Vatican Library ID (former scheme)')]
[('p', 'http://www.wikidata.org/prop/direct/P102'), ('name', 'member of political party')]
[('p', 'http://www.wikidata.org/prop/direct/P103'), ('name', 'native language')]
[('p', 'http:/

In [3]:
# incoming edges to Arthur Conan Doyle
queryString = """
SELECT DISTINCT ?p ?name
WHERE {
   # bind something
    ?sub ?p wd:Q35610 .
   # get the label
   ?p sc:name ?name.
}
LIMIT 200
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P101'), ('name', 'field of work')]
[('p', 'http://www.wikidata.org/prop/direct/P1269'), ('name', 'facet of')]
[('p', 'http://www.wikidata.org/prop/direct/P138'), ('name', 'named after')]
[('p', 'http://www.wikidata.org/prop/direct/P161'), ('name', 'cast member')]
[('p', 'http://www.wikidata.org/prop/direct/P170'), ('name', 'creator')]
[('p', 'http://www.wikidata.org/prop/direct/P180'), ('name', 'depicts')]
[('p', 'http://www.wikidata.org/prop/direct/P1889'), ('name', 'different from')]
[('p', 'http://www.wikidata.org/prop/direct/P22'), ('name', 'father')]
[('p', 'http://www.wikidata.org/prop/direct/P26'), ('name', 'spouse')]
[('p', 'http://www.wikidata.org/prop/direct/P301'), ('name', "category's main topic")]
[('p', 'http://www.wikidata.org/prop/direct/P3342'), ('name', 'significant person')]
[('p', 'http://www.wikidata.org/prop/direct/P40'), ('name', 'child')]
[('p', 'http://www.wikidata.org/prop/direct/P50'), ('name', 'author')]
[(

In [4]:
# subjects with properties "creator","field of work", "characters" or "main subject" and object Arthur Conan Doyle.
# increased the limit to 30
queryString = """
SELECT DISTINCT ?subName ?name
WHERE {
   # bind something
    ?sub ?p wd:Q35610 .
   # get the label
   ?p sc:name ?name.
   ?sub sc:name ?subName
   VALUES ?p {wdt:P170 wdt:P101 wdt:P674 wdt:P921}
}
LIMIT 30

"""

print("Results")
x=run_query(queryString)

Results
[('subName', 'Inspector Lestrade'), ('name', 'creator')]
[('subName', 'Dr. John Watson'), ('name', 'creator')]
[('subName', 'Stamford'), ('name', 'creator')]
[('subName', 'Enoch J. Drebber'), ('name', 'creator')]
[('subName', 'Inspector Gregson'), ('name', 'creator')]
[('subName', 'Sherlock Holmes'), ('name', 'creator')]
[('subName', 'Professor Moriarty'), ('name', 'creator')]
[('subName', 'Mary Morstan'), ('name', 'creator')]
[('subName', 'Edward Malone'), ('name', 'creator')]
[('subName', 'Professor Challenger'), ('name', 'creator')]
[('subName', 'Lord John Roxton'), ('name', 'creator')]
[('subName', 'Mrs. Hudson'), ('name', 'creator')]
[('subName', '221B Baker Street'), ('name', 'creator')]
[('subName', 'Irene Adler'), ('name', 'creator')]
[('subName', 'Baker Street Irregulars'), ('name', 'creator')]
[('subName', 'Inspector MacDonald'), ('name', 'creator')]
[('subName', 'Inspector Bradstreet'), ('name', 'creator')]
[('subName', 'Inspector Hopkins'), ('name', 'creator')]
[('s

In [5]:
# basically the same query as before with limit up to 100
queryString = """
SELECT DISTINCT ?sub ?subName 
WHERE {
   # bind something
   ?sub ?p wd:Q35610 .
   # get the label
   ?sub sc:name ?subName.
   VALUES ?p {wdt:P170 wdt:P101 wdt:P674 wdt:P921}
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('sub', 'http://www.wikidata.org/entity/Q1067723'), ('subName', 'Inspector Lestrade')]
[('sub', 'http://www.wikidata.org/entity/Q187349'), ('subName', 'Dr. John Watson')]
[('sub', 'http://www.wikidata.org/entity/Q20730767'), ('subName', 'Stamford')]
[('sub', 'http://www.wikidata.org/entity/Q20730814'), ('subName', 'Enoch J. Drebber')]
[('sub', 'http://www.wikidata.org/entity/Q3271215'), ('subName', 'Inspector Gregson')]
[('sub', 'http://www.wikidata.org/entity/Q4653'), ('subName', 'Sherlock Holmes')]
[('sub', 'http://www.wikidata.org/entity/Q283111'), ('subName', 'Professor Moriarty')]
[('sub', 'http://www.wikidata.org/entity/Q2731317'), ('subName', 'Mary Morstan')]
[('sub', 'http://www.wikidata.org/entity/Q3719991'), ('subName', 'Edward Malone')]
[('sub', 'http://www.wikidata.org/entity/Q2330719'), ('subName', 'Professor Challenger')]
[('sub', 'http://www.wikidata.org/entity/Q6679455'), ('subName', 'Lord John Roxton')]
[('sub', 'http://www.wikidata.org/entity/Q2456753'), ('su

In [6]:
# FINAL QUERY
# subjects that contain "canon" with incoming edges to Arthur Conan Doyle.
# a piece of work ("canon of Sherlock Holmes") should be directly connected with it's author
queryString = """
SELECT DISTINCT  ?sub ?subName ?p ?name 
WHERE {
   # bind something
   ?sub ?p wd:Q35610 .
   # get the label
   ?p sc:name ?name.
   ?sub sc:name ?subName.
   FILTER(REGEX(?subName, "(?i)canon"))
}
LIMIT 200
"""

print("Results")
x=run_query(queryString)

Results
[('sub', 'http://www.wikidata.org/entity/Q2316684'), ('subName', 'canon of Sherlock Holmes'), ('p', 'http://www.wikidata.org/prop/direct/P50'), ('name', 'author')]
1


> The node we are interested in is "wd:Q2316684" with label "canon of Sherlock Holmes"

## Task 2
Identify the BGP for film

In [7]:
# all properties of the canon of Sherlock Holmes
# The only starting point I have to find the BGP for "film" is either Arthur Conan Doyle or the "canon of Sherlock Holmes"

queryString = """
SELECT DISTINCT ?p ?name
WHERE {
   # bind something
   wd:Q2316684 ?p ?obj .
   # get the label
   ?p sc:name ?name.
}
LIMIT 20
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P136'), ('name', 'genre')]
[('p', 'http://www.wikidata.org/prop/direct/P31'), ('name', 'instance of')]
[('p', 'http://www.wikidata.org/prop/direct/P373'), ('name', 'Commons category')]
[('p', 'http://www.wikidata.org/prop/direct/P4969'), ('name', 'derivative work')]
[('p', 'http://www.wikidata.org/prop/direct/P50'), ('name', 'author')]
[('p', 'http://www.wikidata.org/prop/direct/P527'), ('name', 'has part')]
[('p', 'http://www.wikidata.org/prop/direct/P646'), ('name', 'Freebase ID')]
[('p', 'http://www.wikidata.org/prop/direct/P674'), ('name', 'characters')]
[('p', 'http://www.wikidata.org/prop/direct/P6839'), ('name', 'TV Tropes identifier')]
[('p', 'http://www.wikidata.org/prop/direct/P8895'), ('name', 'All the Tropes identifier')]
[('p', 'http://www.wikidata.org/prop/direct/P910'), ('name', "topic's main category")]
11


In [8]:
# I know there are movies about Sherlock Holmes, so I look for
# "derivative work" (wdt:P4969) of the "Canon of Sherlock Holmes"
queryString = """
SELECT DISTINCT ?obj ?name
WHERE {
   # bind something
   wd:Q2316684 wdt:P4969 ?obj .
   # get the label
   ?obj sc:name ?name.
}
LIMIT 20
"""

print("Results")
x=run_query(queryString)

Results
[('obj', 'http://www.wikidata.org/entity/Q1764019'), ('name', 'Sherlock Holmes')]
[('obj', 'http://www.wikidata.org/entity/Q192837'), ('name', 'Sherlock')]
[('obj', 'http://www.wikidata.org/entity/Q247643'), ('name', 'Elementary')]
[('obj', 'http://www.wikidata.org/entity/Q4523158'), ('name', 'Sherlock Holmes')]
[('obj', 'http://www.wikidata.org/entity/Q200396'), ('name', 'Sherlock Holmes')]
[('obj', 'http://www.wikidata.org/entity/Q1210893'), ('name', 'The Adventures of Sherlock Holmes and Dr. Watson')]
[('obj', 'http://www.wikidata.org/entity/Q387306'), ('name', 'Sherlock Holmes')]
[('obj', 'http://www.wikidata.org/entity/Q243643'), ('name', 'Sherlock Holmes: A Game of Shadows')]
8


In [9]:
# FINAL QUERY
# What are the derivative works "instances of" (wdt:P31") ?
queryString = """
SELECT DISTINCT ?obj ?name
WHERE {
   # bind something
   wd:Q2316684 wdt:P4969/wdt:P31 ?obj .
   # get the label
   ?obj sc:name ?name.
}
LIMIT 20
"""

print("Results")
x=run_query(queryString)

Results
[('obj', 'http://www.wikidata.org/entity/Q24856'), ('name', 'film series')]
[('obj', 'http://www.wikidata.org/entity/Q11424'), ('name', 'film')]
[('obj', 'http://www.wikidata.org/entity/Q5398426'), ('name', 'television series')]
[('obj', 'http://www.wikidata.org/entity/Q1259759'), ('name', 'miniseries')]
4


>Each row represents different instances of derivative work. The node we are interested in is "wd:Q11424" with label "film".\
This query is also the solution for the next task.

## Task 3
Identify the BGP for television series

In [10]:
# FINAL QUERY
# Same query as the previous one.
queryString = """
SELECT DISTINCT ?obj ?name
WHERE {
   # bind something
   wd:Q2316684 wdt:P4969/wdt:P31 ?obj .
   # get the label
   ?obj sc:name ?name.
}
LIMIT 20
"""

print("Results")
x=run_query(queryString)

Results
[('obj', 'http://www.wikidata.org/entity/Q24856'), ('name', 'film series')]
[('obj', 'http://www.wikidata.org/entity/Q11424'), ('name', 'film')]
[('obj', 'http://www.wikidata.org/entity/Q5398426'), ('name', 'television series')]
[('obj', 'http://www.wikidata.org/entity/Q1259759'), ('name', 'miniseries')]
4


>Each row represents different instances of derivative work. The node we are interested in is "wd:Q5398426" with label "television series"

## Task 4
Return the year of publication (take the first one if there are more publication dates) for each film and television series related to the canon of Sherlock Holmes (the result set must be a list of triples film/tv series IRI, label and year).

In [11]:
# Properties of films and tv series related to the canon of Sherlock Holmes
queryString = """
SELECT DISTINCT ?p ?name
WHERE {
   # bind something
   wd:Q2316684 wdt:P4969 ?derivativeWork.
   ?derivativeWork wdt:P31 ?type;
                   ?p ?obj.
   # get the label
   ?p sc:name ?name.
   VALUES ?type {wd:Q5398426 wd:Q11424}
}
LIMIT 200
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P1040'), ('name', 'film editor')]
[('p', 'http://www.wikidata.org/prop/direct/P1113'), ('name', 'number of episodes')]
[('p', 'http://www.wikidata.org/prop/direct/P1237'), ('name', 'Box Office Mojo film ID (former scheme)')]
[('p', 'http://www.wikidata.org/prop/direct/P1258'), ('name', 'Rotten Tomatoes ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1265'), ('name', 'AlloCiné film ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1267'), ('name', 'AlloCiné series ID')]
[('p', 'http://www.wikidata.org/prop/direct/P136'), ('name', 'genre')]
[('p', 'http://www.wikidata.org/prop/direct/P1411'), ('name', 'nominated for')]
[('p', 'http://www.wikidata.org/prop/direct/P1417'), ('name', 'Encyclopædia Britannica Online ID')]
[('p', 'http://www.wikidata.org/prop/direct/P144'), ('name', 'based on')]
[('p', 'http://www.wikidata.org/prop/direct/P1476'), ('name', 'title')]
[('p', 'http://www.wikidata.org/prop/direct/P154'), ('name', 'logo image'

In [12]:
# Check objects with predicate publication date (wdt:P577). But publication date seems to never be used.
# Actually, I am looking for objects that have a label, so excluding strings such as dates. That's why empty result set.
queryString = """
SELECT DISTINCT ?obj ?name
WHERE {
   # bind something
   ?derivativeWork wdt:P577 ?obj.
   # get the label
   ?obj sc:name ?name.
   #?derivativeWork sc:name ?dName.
   #VALUES ?type {wd:Q5398426 wd:Q11424}
}
LIMIT 2
"""

print("Results")
x=run_query(queryString)

Results
Empty


In [13]:
# Derivative work of the canon of Sherlock Holmes
queryString = """
SELECT DISTINCT ?derivativeWork ?derivativeWorkName
WHERE {
   # bind something
   wd:Q2316684 wdt:P4969 ?derivativeWork.
   ?derivativeWork wdt:P31 ?type.
   
   # get the label
   ?derivativeWork sc:name ?derivativeWorkName.
   VALUES ?type {wd:Q5398426 wd:Q11424}
}
LIMIT 20
"""

print("Results")
x=run_query(queryString)

Results
[('derivativeWork', 'http://www.wikidata.org/entity/Q192837'), ('derivativeWorkName', 'Sherlock')]
[('derivativeWork', 'http://www.wikidata.org/entity/Q247643'), ('derivativeWorkName', 'Elementary')]
[('derivativeWork', 'http://www.wikidata.org/entity/Q4523158'), ('derivativeWorkName', 'Sherlock Holmes')]
[('derivativeWork', 'http://www.wikidata.org/entity/Q200396'), ('derivativeWorkName', 'Sherlock Holmes')]
[('derivativeWork', 'http://www.wikidata.org/entity/Q387306'), ('derivativeWorkName', 'Sherlock Holmes')]
[('derivativeWork', 'http://www.wikidata.org/entity/Q243643'), ('derivativeWorkName', 'Sherlock Holmes: A Game of Shadows')]
6


In [14]:
# Properties of Sherlock Holmes: A Game of Shadows
queryString = """
SELECT DISTINCT ?p ?name
WHERE {
   # bind something
   wd:Q243643 ?p ?obj .
   # get the label
   ?p sc:name ?name.
}
LIMIT 2000
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P1040'), ('name', 'film editor')]
[('p', 'http://www.wikidata.org/prop/direct/P1237'), ('name', 'Box Office Mojo film ID (former scheme)')]
[('p', 'http://www.wikidata.org/prop/direct/P1258'), ('name', 'Rotten Tomatoes ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1265'), ('name', 'AlloCiné film ID')]
[('p', 'http://www.wikidata.org/prop/direct/P136'), ('name', 'genre')]
[('p', 'http://www.wikidata.org/prop/direct/P144'), ('name', 'based on')]
[('p', 'http://www.wikidata.org/prop/direct/P1476'), ('name', 'title')]
[('p', 'http://www.wikidata.org/prop/direct/P155'), ('name', 'follows')]
[('p', 'http://www.wikidata.org/prop/direct/P1562'), ('name', 'AllMovie title ID')]
[('p', 'http://www.wikidata.org/prop/direct/P161'), ('name', 'cast member')]
[('p', 'http://www.wikidata.org/prop/direct/P162'), ('name', 'producer')]
[('p', 'http://www.wikidata.org/prop/direct/P1712'), ('name', 'Metacritic ID')]
[('p', 'http://www.wikidata.org/pro

In [15]:
# Publication date of Sherlock Holmes: A Game of Shadows. Empty (Again, I am excluding literals..)
queryString = """
SELECT DISTINCT ?obj ?name
WHERE {
   # bind something
   wd:Q243643 wdt:P577 ?obj .
   # get the label
   ?obj sc:name ?name.
}
LIMIT 2000
"""

print("Results")
x=run_query(queryString)

Results
Empty


In [16]:
# All literals of properties of Sherlock Holmes: A Game of Shadows. 
# Only here I realize using "sc:name" automatically filters out all literals, because they don't have a label
queryString = """
SELECT DISTINCT ?p ?name ?obj 
WHERE {
   # bind something
   wd:Q243643 ?p ?obj.
   # get the label
   ?p sc:name ?name
   FILTER(isLiteral(?obj))
}
LIMIT 200
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P1237'), ('name', 'Box Office Mojo film ID (former scheme)'), ('obj', 'sherlockholmes2')]
[('p', 'http://www.wikidata.org/prop/direct/P1258'), ('name', 'Rotten Tomatoes ID'), ('obj', 'm/sherlock_holmes_a_game_of_shadows')]
[('p', 'http://www.wikidata.org/prop/direct/P1265'), ('name', 'AlloCiné film ID'), ('obj', '173048')]
[('p', 'http://www.wikidata.org/prop/direct/P1476'), ('name', 'title'), ('obj', 'Sherlock Holmes: A Game of Shadows')]
[('p', 'http://www.wikidata.org/prop/direct/P1562'), ('name', 'AllMovie title ID'), ('obj', 'v518386')]
[('p', 'http://www.wikidata.org/prop/direct/P1712'), ('name', 'Metacritic ID'), ('obj', 'movie/sherlock-holmes-a-game-of-shadows')]
[('p', 'http://www.wikidata.org/prop/direct/P1804'), ('name', 'DNF film ID'), ('obj', '75170')]
[('p', 'http://www.wikidata.org/prop/direct/P1874'), ('name', 'Netflix ID'), ('obj', '70202148')]
[('p', 'http://www.wikidata.org/prop/direct/P1970'), ('name', 'MovieMeter 

In [17]:
# publication years of films and tv series derivative from the "canon of Sherlock Holmes"
queryString = """
SELECT DISTINCT ?derivativeWork ?derivativeWorkName (year(?obj) as ?Year)
WHERE {
   # bind something
   wd:Q2316684 wdt:P4969 ?derivativeWork.
   ?derivativeWork wdt:P31 ?type;
                   wdt:P577 ?obj.
                   
    ?derivativeWork sc:name ?derivativeWorkName.

   VALUES ?type {wd:Q5398426 wd:Q11424}
}
LIMIT 200
"""

print("Results")
x=run_query(queryString)

Results
[('derivativeWork', 'http://www.wikidata.org/entity/Q247643'), ('derivativeWorkName', 'Elementary'), ('Year', '2013')]
[('derivativeWork', 'http://www.wikidata.org/entity/Q247643'), ('derivativeWorkName', 'Elementary'), ('Year', '2014')]
[('derivativeWork', 'http://www.wikidata.org/entity/Q200396'), ('derivativeWorkName', 'Sherlock Holmes'), ('Year', '2009')]
[('derivativeWork', 'http://www.wikidata.org/entity/Q200396'), ('derivativeWorkName', 'Sherlock Holmes'), ('Year', '2010')]
[('derivativeWork', 'http://www.wikidata.org/entity/Q243643'), ('derivativeWorkName', 'Sherlock Holmes: A Game of Shadows'), ('Year', '2011')]
5


In [18]:
# publication year of films and tv series derivative from the "canon of Sherlock Holmes"
# retrieving only the most recent year
queryString = """
SELECT DISTINCT ?derivativeWork ?derivativeWorkName (MIN(year(?obj)) as ?Year)
WHERE {
   # bind something
   wd:Q2316684 wdt:P4969 ?derivativeWork.
   ?derivativeWork wdt:P31 ?type;
                   wdt:P577 ?obj.
                   
    ?derivativeWork sc:name ?derivativeWorkName.

   VALUES ?type {wd:Q5398426 wd:Q11424}
}
LIMIT 200
"""

print("Results")
x=run_query(queryString)

Results
[('derivativeWork', 'http://www.wikidata.org/entity/Q200396'), ('derivativeWorkName', 'Sherlock Holmes'), ('Year', '2009')]
[('derivativeWork', 'http://www.wikidata.org/entity/Q247643'), ('derivativeWorkName', 'Elementary'), ('Year', '2013')]
[('derivativeWork', 'http://www.wikidata.org/entity/Q243643'), ('derivativeWorkName', 'Sherlock Holmes: A Game of Shadows'), ('Year', '2011')]
3


In [19]:
# Up to know I used "derivative Work" to get related works. I see if there are films or tv series with edges
# incoming to the "canon of Sherlock Holmes".

queryString = """
SELECT DISTINCT ?video ?videoName ?p ?pName 
WHERE {
   # bind something
   ?video ?p wd:Q2316684.
   ?video wdt:P31 ?type.
                   
    ?p sc:name ?pName.
    ?video sc:name ?videoName. 

   VALUES ?type {wd:Q5398426 wd:Q11424}
}
LIMIT 200
"""

print("Results")
x=run_query(queryString)

Results
[('video', 'http://www.wikidata.org/entity/Q200396'), ('videoName', 'Sherlock Holmes'), ('p', 'http://www.wikidata.org/prop/direct/P144'), ('pName', 'based on')]
[('video', 'http://www.wikidata.org/entity/Q247643'), ('videoName', 'Elementary'), ('p', 'http://www.wikidata.org/prop/direct/P144'), ('pName', 'based on')]
[('video', 'http://www.wikidata.org/entity/Q387306'), ('videoName', 'Sherlock Holmes'), ('p', 'http://www.wikidata.org/prop/direct/P144'), ('pName', 'based on')]
[('video', 'http://www.wikidata.org/entity/Q192837'), ('videoName', 'Sherlock'), ('p', 'http://www.wikidata.org/prop/direct/P144'), ('pName', 'based on')]
[('video', 'http://www.wikidata.org/entity/Q243643'), ('videoName', 'Sherlock Holmes: A Game of Shadows'), ('p', 'http://www.wikidata.org/prop/direct/P144'), ('pName', 'based on')]
[('video', 'http://www.wikidata.org/entity/Q4523158'), ('videoName', 'Sherlock Holmes'), ('p', 'http://www.wikidata.org/prop/direct/P144'), ('pName', 'based on')]
[('video', '

In [20]:
# Up to know I used "derivative Work" to get related works. I see if there are films or tv series with edges
# incoming to the "canon of Sherlock Holmes" and their release year.

queryString = """
SELECT DISTINCT ?video ?videoName ?p ?pName (MIN(year(?publicationDate)) as ?Year)
WHERE {
   # bind something
   ?video ?p wd:Q2316684.
   ?video wdt:P31 ?type;
          wdt:P577 ?publicationDate.
                   
    ?p sc:name ?pName.
    ?video sc:name ?videoName. 

   VALUES ?type {wd:Q5398426 wd:Q11424}
}
LIMIT 200
"""

print("Results")
x=run_query(queryString)

Results
[('video', 'http://www.wikidata.org/entity/Q200396'), ('videoName', 'Sherlock Holmes'), ('p', 'http://www.wikidata.org/prop/direct/P144'), ('pName', 'based on'), ('Year', '2009')]
[('video', 'http://www.wikidata.org/entity/Q604180'), ('videoName', 'The Private Life of Sherlock Holmes'), ('p', 'http://www.wikidata.org/prop/direct/P941'), ('pName', 'inspired by'), ('Year', '1970')]
[('video', 'http://www.wikidata.org/entity/Q247643'), ('videoName', 'Elementary'), ('p', 'http://www.wikidata.org/prop/direct/P144'), ('pName', 'based on'), ('Year', '2013')]
[('video', 'http://www.wikidata.org/entity/Q243643'), ('videoName', 'Sherlock Holmes: A Game of Shadows'), ('p', 'http://www.wikidata.org/prop/direct/P144'), ('pName', 'based on'), ('Year', '2011')]
4


In [21]:
# FINAL QUERY
# publication year of films and tv series related to the "canon of Sherlock Holmes"
# retrieving only the most recent year
queryString = """
SELECT DISTINCT (?derivativeWork as ?relatedWork) (?derivativeWorkName as ?relatedWorkName) (MIN(year(?obj)) as ?Year)
WHERE {{
   # bind something
   wd:Q2316684 wdt:P4969 ?derivativeWork.
   ?derivativeWork wdt:P31 ?type;
                   wdt:P577 ?obj.
                   
    ?derivativeWork sc:name ?derivativeWorkName.

   VALUES ?type {wd:Q5398426 wd:Q11424}}
   UNION{
      # bind something
   ?derivativeWork ?p wd:Q2316684.
   ?derivativeWork wdt:P31 ?type;
          wdt:P577 ?obj.
                   
     ?derivativeWork sc:name ?derivativeWorkName. 

   VALUES ?type {wd:Q5398426 wd:Q11424}
   }
}
LIMIT 200
"""

print("Results")
x=run_query(queryString)

Results
[('relatedWork', 'http://www.wikidata.org/entity/Q604180'), ('relatedWorkName', 'The Private Life of Sherlock Holmes'), ('Year', '1970')]
[('relatedWork', 'http://www.wikidata.org/entity/Q200396'), ('relatedWorkName', 'Sherlock Holmes'), ('Year', '2009')]
[('relatedWork', 'http://www.wikidata.org/entity/Q247643'), ('relatedWorkName', 'Elementary'), ('Year', '2013')]
[('relatedWork', 'http://www.wikidata.org/entity/Q243643'), ('relatedWorkName', 'Sherlock Holmes: A Game of Shadows'), ('Year', '2011')]
4


>Results in the format "Related Work IRI" - "Related Work Label" - "Year of publication"

## Task 5
Find who acted in more than one film or tv series related to the canon of Sherlock Holmes (the result set must be a list of triples actor/actress IRI, label and #film).

In [22]:
# Cast member (wdt:P161) found with CTRL+F in the notebook. 
# results seems to have only actors of the two major latest movies (the ones with Jude Law)..
queryString = """
SELECT DISTINCT ?castMember ?castMemberName (COUNT(DISTINCT ?derivativeWork) as ?film)
WHERE { 
   # bind something
   wd:Q2316684 wdt:P4969 ?derivativeWork.
   ?derivativeWork wdt:P31 ?type;
                   wdt:P161 ?castMember.
   
   # get the label
   ?derivativeWork sc:name ?derivativeWorkName.
   ?castMember sc:name ?castMemberName.
   VALUES ?type {wd:Q5398426 wd:Q11424}
   
}
GROUP BY ?castMember ?castMemberName 
HAVING (COUNT( DISTINCT ?derivativeWork) > 1)

"""

print("Results")
x=run_query(queryString)

Results
[('castMember', 'http://www.wikidata.org/entity/Q1102002'), ('castMemberName', 'Clive Russell'), ('film', '2')]
[('castMember', 'http://www.wikidata.org/entity/Q190386'), ('castMemberName', 'Rachel McAdams'), ('film', '2')]
[('castMember', 'http://www.wikidata.org/entity/Q45647'), ('castMemberName', 'Eddie Marsan'), ('film', '2')]
[('castMember', 'http://www.wikidata.org/entity/Q440926'), ('castMemberName', 'Geraldine James'), ('film', '2')]
[('castMember', 'http://www.wikidata.org/entity/Q165219'), ('castMemberName', 'Robert Downey Jr.'), ('film', '2')]
[('castMember', 'http://www.wikidata.org/entity/Q160432'), ('castMemberName', 'Jude Law'), ('film', '2')]
[('castMember', 'http://www.wikidata.org/entity/Q236167'), ('castMemberName', 'Kelly Reilly'), ('film', '2')]
7


In [23]:
# looking if there are other ways to retrieve cast members, starting from "Sherlock" (wd:Q192837)
queryString = """
SELECT DISTINCT ?edge ?edgeName 
WHERE {{
   # bind something
   wd:Q192837 ?edge ?object.
   # get the label
   ?edge sc:name ?edgeName.
   
   FILTER (!isLiteral(?object))
   }UNION{
   ?subject ?edge wd:Q192837.
   ?edge sc:name ?edgeName
   }
}
LIMIT 200
"""

print("Results")
x=run_query(queryString)

Results
[('edge', 'http://www.wikidata.org/prop/direct/P136'), ('edgeName', 'genre')]
[('edge', 'http://www.wikidata.org/prop/direct/P1411'), ('edgeName', 'nominated for')]
[('edge', 'http://www.wikidata.org/prop/direct/P144'), ('edgeName', 'based on')]
[('edge', 'http://www.wikidata.org/prop/direct/P154'), ('edgeName', 'logo image')]
[('edge', 'http://www.wikidata.org/prop/direct/P161'), ('edgeName', 'cast member')]
[('edge', 'http://www.wikidata.org/prop/direct/P162'), ('edgeName', 'producer')]
[('edge', 'http://www.wikidata.org/prop/direct/P166'), ('edgeName', 'award received')]
[('edge', 'http://www.wikidata.org/prop/direct/P170'), ('edgeName', 'creator')]
[('edge', 'http://www.wikidata.org/prop/direct/P18'), ('edgeName', 'image')]
[('edge', 'http://www.wikidata.org/prop/direct/P1811'), ('edgeName', 'list of episodes')]
[('edge', 'http://www.wikidata.org/prop/direct/P1881'), ('edgeName', 'list of characters')]
[('edge', 'http://www.wikidata.org/prop/direct/P1889'), ('edgeName', 'di

In [24]:
# looking if there are other ways to retrieve cast members, starting from "Sherlock"(wd:Q192837),
# printing subjects and objects with some specific predicates
# after reading the results it seems that actors are only retrieved with "cast mamber" (wdt:P161)

# This is an inefficient query, because is checking the same edges to be either outgoing or incoming. 
# To make it more efficient I would need use the appropriate VALUES for incoming/outgoing edges 

queryString = """
SELECT DISTINCT ?edge ?edgeName ?iri ?iriName
WHERE {{
   # bind something
   wd:Q192837 ?edge ?iri.
   # get the label
   ?edge sc:name ?edgeName.
   ?iri sc:name ?iriName.
   VALUES ?edge {wdt:P161 wdt:P1881 wdt:P674 wdt:P179 wdt:P527}
   #FILTER (!isLiteral(?object))
   }UNION{
   ?iri ?edge wd:Q192837.
   ?edge sc:name ?edgeName.
   ?iri sc:name ?iriName.
   VALUES ?edge {wdt:P161 wdt:P1881 wdt:P674 wdt:P179 wdt:P527}
   }
}
LIMIT 200
"""

print("Results")
x=run_query(queryString)

Results
[('edge', 'http://www.wikidata.org/prop/direct/P161'), ('edgeName', 'cast member'), ('iri', 'http://www.wikidata.org/entity/Q560286'), ('iriName', 'Mark Gatiss')]
[('edge', 'http://www.wikidata.org/prop/direct/P161'), ('edgeName', 'cast member'), ('iri', 'http://www.wikidata.org/entity/Q472216'), ('iriName', 'Rupert Graves')]
[('edge', 'http://www.wikidata.org/prop/direct/P161'), ('edgeName', 'cast member'), ('iri', 'http://www.wikidata.org/entity/Q309486'), ('iriName', 'Martin Freeman')]
[('edge', 'http://www.wikidata.org/prop/direct/P161'), ('edgeName', 'cast member'), ('iri', 'http://www.wikidata.org/entity/Q1319582'), ('iriName', 'Una Stubbs')]
[('edge', 'http://www.wikidata.org/prop/direct/P161'), ('edgeName', 'cast member'), ('iri', 'http://www.wikidata.org/entity/Q507322'), ('iriName', 'Andrew Scott')]
[('edge', 'http://www.wikidata.org/prop/direct/P161'), ('edgeName', 'cast member'), ('iri', 'http://www.wikidata.org/entity/Q244674'), ('iriName', 'Benedict Cumberbatch')]

In [25]:
# Same as before but trying to distinguish between incoming/outgoing edges.
# It's longer to write
queryString = """
SELECT DISTINCT ?incomingEdge ?incomingEdgeName ?outgoingEdge ?outgoingEdgeName
WHERE {{
   # bind something
   wd:Q192837 ?outgoingEdge?object.
   # get the label
   ?outgoingEdge sc:name ?outgoingEdgeName.
   
   FILTER (!isLiteral(?object)) # I don't really need this
   }UNION{
   ?subject ?incomingEdge wd:Q192837.
   ?incomingEdge sc:name ?incomingEdgeName
   }
}
LIMIT 200
"""

print("Results")
x=run_query(queryString)

Results
[('outgoingEdge', 'http://www.wikidata.org/prop/direct/P136'), ('outgoingEdgeName', 'genre')]
[('outgoingEdge', 'http://www.wikidata.org/prop/direct/P1411'), ('outgoingEdgeName', 'nominated for')]
[('outgoingEdge', 'http://www.wikidata.org/prop/direct/P144'), ('outgoingEdgeName', 'based on')]
[('outgoingEdge', 'http://www.wikidata.org/prop/direct/P154'), ('outgoingEdgeName', 'logo image')]
[('outgoingEdge', 'http://www.wikidata.org/prop/direct/P161'), ('outgoingEdgeName', 'cast member')]
[('outgoingEdge', 'http://www.wikidata.org/prop/direct/P162'), ('outgoingEdgeName', 'producer')]
[('outgoingEdge', 'http://www.wikidata.org/prop/direct/P166'), ('outgoingEdgeName', 'award received')]
[('outgoingEdge', 'http://www.wikidata.org/prop/direct/P170'), ('outgoingEdgeName', 'creator')]
[('outgoingEdge', 'http://www.wikidata.org/prop/direct/P18'), ('outgoingEdgeName', 'image')]
[('outgoingEdge', 'http://www.wikidata.org/prop/direct/P1811'), ('outgoingEdgeName', 'list of episodes')]
[('o

In [26]:
# looking if there are other ways to retrieve cast members, starting from "Sherlock"(wd:Q192837),
# printing subjects and objects with some specific predicates
# after reading the results it seems that actors are only retrieved with "cast mamber" (wdt:P161)

# Same as before, but differentiating between incoming/outgoing edges

queryString = """
SELECT DISTINCT ?edge ?edgeName ?iri ?iriName
WHERE {{
   # bind something
   wd:Q192837 ?edge ?iri.
   # get the label
   ?edge sc:name ?edgeName.
   ?iri sc:name ?iriName.
   VALUES ?edge {wdt:P161 wdt:P1881 wdt:P674 wdt:P527}
   #FILTER (!isLiteral(?object))
   }UNION{
   ?iri ?edge wd:Q192837.
   ?edge sc:name ?edgeName.
   ?iri sc:name ?iriName.
   VALUES ?edge {wdt:P527}
   }
}
LIMIT 200
"""

print("Results")
x=run_query(queryString)

Results
[('edge', 'http://www.wikidata.org/prop/direct/P527'), ('edgeName', 'has part'), ('iri', 'http://www.wikidata.org/entity/Q28101242'), ('iriName', 'The Six Thatchers')]
[('edge', 'http://www.wikidata.org/prop/direct/P527'), ('edgeName', 'has part'), ('iri', 'http://www.wikidata.org/entity/Q28128648'), ('iriName', 'The Final Problem')]
[('edge', 'http://www.wikidata.org/prop/direct/P527'), ('edgeName', 'has part'), ('iri', 'http://www.wikidata.org/entity/Q55827477'), ('iriName', 'Sherlock, season 4')]
[('edge', 'http://www.wikidata.org/prop/direct/P527'), ('edgeName', 'has part'), ('iri', 'http://www.wikidata.org/entity/Q28128111'), ('iriName', 'The Lying Detective')]
[('edge', 'http://www.wikidata.org/prop/direct/P161'), ('edgeName', 'cast member'), ('iri', 'http://www.wikidata.org/entity/Q560286'), ('iriName', 'Mark Gatiss')]
[('edge', 'http://www.wikidata.org/prop/direct/P161'), ('edgeName', 'cast member'), ('iri', 'http://www.wikidata.org/entity/Q472216'), ('iriName', 'Rupert

In [27]:
# Including all derivative works, not only tv series and films. But, same result as before. 
queryString = """
SELECT DISTINCT ?castMember ?castMemberName (COUNT(DISTINCT ?derivativeWork) as ?film)
WHERE { 
   # bind something
   wd:Q2316684 wdt:P4969 ?derivativeWork.
   ?derivativeWork wdt:P31 ?type;
                   wdt:P161 ?castMember.
   
   # get the label
   ?derivativeWork sc:name ?derivativeWorkName.
   ?castMember sc:name ?castMemberName.
   #VALUES ?type {wd:Q5398426 wd:Q11424}
   
}
GROUP BY ?castMember ?castMemberName 
HAVING (COUNT( DISTINCT ?derivativeWork) > 1)

"""

print("Results")
x=run_query(queryString)

Results
[('castMember', 'http://www.wikidata.org/entity/Q1102002'), ('castMemberName', 'Clive Russell'), ('film', '2')]
[('castMember', 'http://www.wikidata.org/entity/Q190386'), ('castMemberName', 'Rachel McAdams'), ('film', '2')]
[('castMember', 'http://www.wikidata.org/entity/Q45647'), ('castMemberName', 'Eddie Marsan'), ('film', '2')]
[('castMember', 'http://www.wikidata.org/entity/Q440926'), ('castMemberName', 'Geraldine James'), ('film', '2')]
[('castMember', 'http://www.wikidata.org/entity/Q165219'), ('castMemberName', 'Robert Downey Jr.'), ('film', '2')]
[('castMember', 'http://www.wikidata.org/entity/Q160432'), ('castMemberName', 'Jude Law'), ('film', '2')]
[('castMember', 'http://www.wikidata.org/entity/Q236167'), ('castMemberName', 'Kelly Reilly'), ('film', '2')]
7


In [28]:
# FINAL QUERY
# Same as the first query for this task but considering also films or tv series with edges
# incoming to the "canon of Sherlock Holmes".
queryString = """
SELECT DISTINCT ?castMember ?castMemberName (COUNT(DISTINCT ?derivativeWork) as ?film)
WHERE { {
   # bind something
   wd:Q2316684 wdt:P4969 ?derivativeWork.
   ?derivativeWork wdt:P31 ?type;
                   wdt:P161 ?castMember.
   
   # get the label
   ?derivativeWork sc:name ?derivativeWorkName.
   ?castMember sc:name ?castMemberName.
   VALUES ?type {wd:Q5398426 wd:Q11424}}
   UNION {
      # bind something
   ?derivativeWork ?p wd:Q2316684.
   ?derivativeWork wdt:P31 ?type;
          wdt:P161 ?castMember.
                   
     # get the label
   ?derivativeWork sc:name ?derivativeWorkName.
   ?castMember sc:name ?castMemberName.
   VALUES ?type {wd:Q5398426 wd:Q11424}
   }
   
   }
   

GROUP BY ?castMember ?castMemberName 
HAVING (COUNT( DISTINCT ?derivativeWork) > 1)

"""

print("Results")
x=run_query(queryString)

Results
[('castMember', 'http://www.wikidata.org/entity/Q1102002'), ('castMemberName', 'Clive Russell'), ('film', '2')]
[('castMember', 'http://www.wikidata.org/entity/Q190386'), ('castMemberName', 'Rachel McAdams'), ('film', '2')]
[('castMember', 'http://www.wikidata.org/entity/Q45647'), ('castMemberName', 'Eddie Marsan'), ('film', '2')]
[('castMember', 'http://www.wikidata.org/entity/Q440926'), ('castMemberName', 'Geraldine James'), ('film', '2')]
[('castMember', 'http://www.wikidata.org/entity/Q165219'), ('castMemberName', 'Robert Downey Jr.'), ('film', '2')]
[('castMember', 'http://www.wikidata.org/entity/Q160432'), ('castMemberName', 'Jude Law'), ('film', '2')]
[('castMember', 'http://www.wikidata.org/entity/Q236167'), ('castMemberName', 'Kelly Reilly'), ('film', '2')]
7


>Results are in the format "Actor IRI" - "Actor Label" - "Number of films"

## Task 6
Identify the BGP for Primetime Emmy Award

In [29]:
# From Wikipedia: "presented in recognition of excellence in American primetime television programming".
# with CTRL+F found "award received" (wdt:P166) in this notebook. (I was looking for "award", substring of the requested BGP)
queryString = """
SELECT DISTINCT ?obj ?name
WHERE {
   # bind something
   wd:Q2316684 wdt:P4969 ?derivativeWork.
   ?derivativeWork wdt:P31 ?type;
                   wdt:P166 ?obj.
   # get the label
   ?obj sc:name ?name.
   VALUES ?type {wd:Q5398426 wd:Q11424}
}
LIMIT 200
"""

print("Results")
x=run_query(queryString)

Results
[('obj', 'http://www.wikidata.org/entity/Q838121'), ('name', 'Peabody Awards')]
[('obj', 'http://www.wikidata.org/entity/Q7243506'), ('name', 'Primetime Emmy Award for Outstanding Made for Television Movie')]
[('obj', 'http://www.wikidata.org/entity/Q945887'), ('name', 'Primetime Emmy Award for Outstanding Supporting Actor in a Miniseries or a Movie')]
[('obj', 'http://www.wikidata.org/entity/Q989453'), ('name', 'Primetime Emmy Award for Outstanding Lead Actor in a Miniseries or a Movie')]
[('obj', 'http://www.wikidata.org/entity/Q17986500'), ('name', 'Primetime Emmy Award for Outstanding Cinematography for a Miniseries or Movie')]
[('obj', 'http://www.wikidata.org/entity/Q2925700'), ('name', 'British Academy Television Award for Best Supporting Actor')]
[('obj', 'http://www.wikidata.org/entity/Q7243513'), ('name', 'Primetime Emmy Award for Outstanding Special Visual Effects')]
[('obj', 'http://www.wikidata.org/entity/Q18154853'), ('name', 'Teen Choice Award for Choice TV Actre

In [30]:
# FINAL QUERY
# IRIs connected with Primetime Emmy Award for Outstanding Made for Television Movie
queryString = """
SELECT DISTINCT ?iri ?iriName 
WHERE {{
   # bind something
   wd:Q7243506 ?edge ?iri.
   # get the label
   ?iri sc:name ?iriName.
   }UNION{
   ?iri ?edge wd:Q7243506.
   ?iri sc:name ?iriName
   }
}
LIMIT 200
"""

print("Results")
x=run_query(queryString)


Results
[('iri', 'http://www.wikidata.org/entity/Q30'), ('iriName', 'United States of America')]
[('iri', 'http://www.wikidata.org/entity/Q1044427'), ('iriName', 'Primetime Emmy Award')]
[('iri', 'http://www.wikidata.org/entity/Q38033430'), ('iriName', 'class of award')]
[('iri', 'http://www.wikidata.org/entity/Q558112'), ('iriName', 'Black Mirror')]
[('iri', 'http://www.wikidata.org/entity/Q229305'), ('iriName', 'Joan Allen')]
[('iri', 'http://www.wikidata.org/entity/Q1159206'), ('iriName', 'Dana Brunetti')]
[('iri', 'http://www.wikidata.org/entity/Q1033016'), ('iriName', 'Halle Berry')]
[('iri', 'http://www.wikidata.org/entity/Q300341'), ('iriName', 'A Bright Shining Lie')]
[('iri', 'http://www.wikidata.org/entity/Q560286'), ('iriName', 'Mark Gatiss')]
[('iri', 'http://www.wikidata.org/entity/Q3133366'), ('iriName', 'Sue Vertue')]
[('iri', 'http://www.wikidata.org/entity/Q764812'), ('iriName', 'August Wilson')]
[('iri', 'http://www.wikidata.org/entity/Q315808'), ('iriName', 'Neil Sim

>The node we are interested in is wd:Q1044427 with label "Primetime Emmy Award"

## Task 7
Return all the types of Primetime Emmy Award (the result set must be a list of couples award IRI and label).

In [31]:
# Retrieving predicates between Primetime Emmy Award for Outstanding Made for Television Movie 
# and Primetime Emmy Award -> "instance of" (wdt:P31)
queryString = """
SELECT DISTINCT ?edge ?edgeName ?iri ?iriName 
WHERE {{
   # bind something
   wd:Q7243506 ?edge ?iri.
   # get the label
   ?iri sc:name ?iriName.
   ?edge sc:name ?edgeName.

   }UNION{
   ?iri ?edge wd:Q7243506.
   ?iri sc:name ?iriName.
   ?edge sc:name ?edgeName.
   }
}
LIMIT 2
"""

print("Results")
x=run_query(queryString)

Results
[('edge', 'http://www.wikidata.org/prop/direct/P17'), ('edgeName', 'country'), ('iri', 'http://www.wikidata.org/entity/Q30'), ('iriName', 'United States of America')]
[('edge', 'http://www.wikidata.org/prop/direct/P31'), ('edgeName', 'instance of'), ('iri', 'http://www.wikidata.org/entity/Q1044427'), ('iriName', 'Primetime Emmy Award')]
2


In [32]:
# FINAL QUERY
# All instances of Primetime Emmy Award
queryString = """
SELECT DISTINCT ?award ?name
WHERE {
   # bind something
   ?award wdt:P31 wd:Q1044427.
   # get the label
   ?award sc:name ?name.
}
LIMIT 200
"""

print("Results")
x=run_query(queryString)

Results
[('award', 'http://www.wikidata.org/entity/Q18153303'), ('name', 'Primetime Emmy Award for Outstanding Unstructured Reality Program')]
[('award', 'http://www.wikidata.org/entity/Q28445729'), ('name', 'Primetime Emmy Award for Outstanding Actress in a Short Form Comedy or Drama Series')]
[('award', 'http://www.wikidata.org/entity/Q989439'), ('name', 'Primetime Emmy Award for Outstanding Lead Actor in a Drama Series')]
[('award', 'http://www.wikidata.org/entity/Q2604800'), ('name', 'Primetime Emmy Award for Outstanding Guest Actor in a Drama Series')]
[('award', 'http://www.wikidata.org/entity/Q7243506'), ('name', 'Primetime Emmy Award for Outstanding Made for Television Movie')]
[('award', 'http://www.wikidata.org/entity/Q1277961'), ('name', 'Primetime Emmy Award for Outstanding Directing for a Comedy Series')]
[('award', 'http://www.wikidata.org/entity/Q3123491'), ('name', 'Primetime Emmy Award for Outstanding Writing for a Drama Series')]
[('award', 'http://www.wikidata.org/en

> Each row is a type of Primetime Emmy Award in the format "Award IRI" - "Award Label"

## Task 8
For each film and television series related to the canon of Sherlock Holmes, return the number of Primetime Emmy Awards they won (return also the objects who did not win an Award) (the result set must be a list of triples film/tv series IRI, label and #award).

In [33]:
# Number of awards per derivative work 
queryString = """
SELECT DISTINCT ?derivativeWork ?derivativeWorkName (COUNT(DISTINCT ?premio) as ?award)
WHERE {
   # bind something
   wd:Q2316684 wdt:P4969 ?derivativeWork.
   ?derivativeWork wdt:P31 ?type.
   VALUES ?type {wd:Q5398426 wd:Q11424}
   
   OPTIONAL{
       ?derivativeWork wdt:P166 ?premio.
       ?premio wdt:P31  wd:Q1044427.
       }
       
    # get the label
    ?derivativeWork sc:name ?derivativeWorkName.
    
   
}
GROUP BY  ?derivativeWork ?derivativeWorkName
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('derivativeWork', 'http://www.wikidata.org/entity/Q192837'), ('derivativeWorkName', 'Sherlock'), ('award', '8')]
[('derivativeWork', 'http://www.wikidata.org/entity/Q4523158'), ('derivativeWorkName', 'Sherlock Holmes'), ('award', '0')]
[('derivativeWork', 'http://www.wikidata.org/entity/Q387306'), ('derivativeWorkName', 'Sherlock Holmes'), ('award', '0')]
[('derivativeWork', 'http://www.wikidata.org/entity/Q247643'), ('derivativeWorkName', 'Elementary'), ('award', '0')]
[('derivativeWork', 'http://www.wikidata.org/entity/Q200396'), ('derivativeWorkName', 'Sherlock Holmes'), ('award', '0')]
[('derivativeWork', 'http://www.wikidata.org/entity/Q243643'), ('derivativeWorkName', 'Sherlock Holmes: A Game of Shadows'), ('award', '0')]
6


In [34]:
# Including all derivative works, not only tv series and films. But, same result as before. 
queryString = """
SELECT DISTINCT ?derivativeWork ?derivativeWorkName (COUNT( DISTINCT ?premio) as ?award)
WHERE {
   # bind something
   wd:Q2316684 wdt:P4969 ?derivativeWork.
   
   OPTIONAL{
   ?derivativeWork wdt:P166 ?premio.
   ?premio wdt:P31  wd:Q1044427.
   }
   # get the label

   ?derivativeWork sc:name ?derivativeWorkName.
}
GROUP BY  ?derivativeWork ?derivativeWorkName
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('derivativeWork', 'http://www.wikidata.org/entity/Q200396'), ('derivativeWorkName', 'Sherlock Holmes'), ('award', '0')]
[('derivativeWork', 'http://www.wikidata.org/entity/Q4523158'), ('derivativeWorkName', 'Sherlock Holmes'), ('award', '0')]
[('derivativeWork', 'http://www.wikidata.org/entity/Q1764019'), ('derivativeWorkName', 'Sherlock Holmes'), ('award', '0')]
[('derivativeWork', 'http://www.wikidata.org/entity/Q1210893'), ('derivativeWorkName', 'The Adventures of Sherlock Holmes and Dr. Watson'), ('award', '0')]
[('derivativeWork', 'http://www.wikidata.org/entity/Q192837'), ('derivativeWorkName', 'Sherlock'), ('award', '8')]
[('derivativeWork', 'http://www.wikidata.org/entity/Q243643'), ('derivativeWorkName', 'Sherlock Holmes: A Game of Shadows'), ('award', '0')]
[('derivativeWork', 'http://www.wikidata.org/entity/Q247643'), ('derivativeWorkName', 'Elementary'), ('award', '0')]
[('derivativeWork', 'http://www.wikidata.org/entity/Q387306'), ('derivativeWorkName', 'Sherlock

In [35]:
# wrong query
queryString = """
SELECT DISTINCT ?derivativeWork ?derivativeWorkName (COUNT( DISTINCT ?premio) as ?award)
WHERE {
   # bind something
   wd:Q2316684 wdt:P4969 ?derivativeWork.
   ?anotherWork wdt:P94 wd:Q2316684. #adding also the works inspired by the canon of Sherlock Holmes
   OPTIONAL{
   ?anotherWork wdt:P166 ?premio.
   ?premio wdt:P31  wd:Q1044427.
   }
   OPTIONAL{
   ?derivativeWork wdt:P166 ?premio.
   ?premio wdt:P31  wd:Q1044427.
   }
   # get the label

   ?derivativeWork sc:name ?derivativeWorkName.
   ?anotherWork sc:name ?anotherWorkName.
}
GROUP BY  ?derivativeWork ?derivativeWorkName
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
Empty


In [36]:
# Number of awards per derivative work. Not working because I am using wdt:P94 instead of wdt:P941.
queryString = """
SELECT DISTINCT ?derivativeWork ?derivativeWorkName (COUNT(DISTINCT ?premio) as ?award)
WHERE {{
   # bind something
   wd:Q2316684 wdt:P4969 ?derivativeWork.
   ?derivativeWork wdt:P31 ?type.
   VALUES ?type {wd:Q5398426 wd:Q11424}
   
   OPTIONAL{
       ?derivativeWork wdt:P166 ?premio.
       ?premio wdt:P31  wd:Q1044427.
       }
       
    # get the label
    ?derivativeWork sc:name ?derivativeWorkName.
    
    }UNION {
       # bind something
   ?derivativeWork wdt:P94 wd:Q2316684 . #don't know why this does not return "The Private Life of Sherlock Holmes"
   ?derivativeWork wdt:P31 ?type.
   VALUES ?type {wd:Q5398426 wd:Q11424}
   
   OPTIONAL{
       ?derivativeWork wdt:P166 ?premio.
       ?premio wdt:P31  wd:Q1044427.
       }
       
    # get the label
    ?derivativeWork sc:name ?derivativeWorkName.
    }
   
}
GROUP BY  ?derivativeWork ?derivativeWorkName
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('derivativeWork', 'http://www.wikidata.org/entity/Q192837'), ('derivativeWorkName', 'Sherlock'), ('award', '8')]
[('derivativeWork', 'http://www.wikidata.org/entity/Q4523158'), ('derivativeWorkName', 'Sherlock Holmes'), ('award', '0')]
[('derivativeWork', 'http://www.wikidata.org/entity/Q387306'), ('derivativeWorkName', 'Sherlock Holmes'), ('award', '0')]
[('derivativeWork', 'http://www.wikidata.org/entity/Q247643'), ('derivativeWorkName', 'Elementary'), ('award', '0')]
[('derivativeWork', 'http://www.wikidata.org/entity/Q200396'), ('derivativeWorkName', 'Sherlock Holmes'), ('award', '0')]
[('derivativeWork', 'http://www.wikidata.org/entity/Q243643'), ('derivativeWorkName', 'Sherlock Holmes: A Game of Shadows'), ('award', '0')]
6


In [37]:
# FINAL QUERY
# Number of awards per related film and television series
queryString = """
SELECT DISTINCT (?derivativeWork as ?relatedWork) (?derivativeWorkName as ?relatedWorkName) (COUNT(DISTINCT ?premio) as ?award)
WHERE {{
   # bind something
   wd:Q2316684 wdt:P4969 ?derivativeWork.
   ?derivativeWork wdt:P31 ?type.
   VALUES ?type {wd:Q5398426 wd:Q11424}
       
    # get the label
    ?derivativeWork sc:name ?derivativeWorkName.
    
    }UNION {
       # bind something
   ?derivativeWork ?p wd:Q2316684 . #Including "The Private Life of Sherlock Holmes"
   ?derivativeWork wdt:P31 ?type.
   VALUES ?type {wd:Q5398426 wd:Q11424}
          
    # get the label
    ?derivativeWork sc:name ?derivativeWorkName.
    }
     OPTIONAL{
       ?derivativeWork wdt:P166 ?premio.
       ?premio wdt:P31  wd:Q1044427.
       }
   
}
GROUP BY  ?derivativeWork ?derivativeWorkName
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('relatedWork', 'http://www.wikidata.org/entity/Q604180'), ('relatedWorkName', 'The Private Life of Sherlock Holmes'), ('award', '0')]
[('relatedWork', 'http://www.wikidata.org/entity/Q192837'), ('relatedWorkName', 'Sherlock'), ('award', '8')]
[('relatedWork', 'http://www.wikidata.org/entity/Q4523158'), ('relatedWorkName', 'Sherlock Holmes'), ('award', '0')]
[('relatedWork', 'http://www.wikidata.org/entity/Q387306'), ('relatedWorkName', 'Sherlock Holmes'), ('award', '0')]
[('relatedWork', 'http://www.wikidata.org/entity/Q247643'), ('relatedWorkName', 'Elementary'), ('award', '0')]
[('relatedWork', 'http://www.wikidata.org/entity/Q200396'), ('relatedWorkName', 'Sherlock Holmes'), ('award', '0')]
[('relatedWork', 'http://www.wikidata.org/entity/Q243643'), ('relatedWorkName', 'Sherlock Holmes: A Game of Shadows'), ('award', '0')]
7


>Results in the format "Related Work IRI" - "Related Work Label" - "number of awards"